# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning', 'R. E. Hviding']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']
Y. Wu  ->  Y. Wu  |  ['Y. Wu']


Arxiv has 66 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2601.10795
extracting tarball to tmp_2601.10795...

 done.


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2601.10795/AandA.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authors' from 'tmp_2601.10795/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


Error retrieving bib data for #1: '#1'
Retrieving document from  https://arxiv.org/e-print/2601.11127
extracting tarball to tmp_2601.11127... done.


E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']


Retrieving document from  https://arxiv.org/e-print/2601.11280
extracting tarball to tmp_2601.11280...

 done.
Retrieving document from  https://arxiv.org/e-print/2601.11469
extracting tarball to tmp_2601.11469... done.


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2601.11469/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'dis' from 'tmp_2601.11469/dis.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'lens' from 'tmp_2601.11469/lens.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: Late

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.10795-b31b1b.svg)](https://arxiv.org/abs/2601.10795) | **Euclid: Galaxy SED reconstruction in the PHZ processing function: impact on the PSF and the role of medium-band filters**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2026-01-19*|
|*Comments*| *17 pages, 10 figures, 2 tables, including appendices*|
|**Abstract**|            Weak lensing surveys require accurate correction for the point spread function (PSF) when measuring galaxy shapes. For a diffraction-limited PSF, as arises in space-based missions, this correction depends on each galaxy SED. In the Euclid mission, galaxy SED reconstruction, a tasks of the photometric-redshift processing function (PHZ PF), relies on broad- and medium-band ancillary photometry. The limited wavelength sampling of the Euclid VIS passband and signal-to-noise ratio may affect the reconstruction accuracy and translate into biases in the weak lensing measurements. In this study, we present the methodology, which is employed in the Euclid PHZ PF, for reconstructing galaxy SEDs at 55 wavelengths, sampling the VIS passband every 10 nm, and we assess whether it fulfils the accuracy requirements imposed on the Euclid PSF model. We employ both physics- and data-driven methods, focusing on a new approach of template-based flux correction and Gaussian processes, and we introduce an SED metric whose bias propagates into PSF quadrupole moment errors. Our findings demonstrate that Gaussian processes and template fitting meet the requirements only in specific, but complementary, redshift intervals. We therefore propose a hybrid approach, which leverages both methods. This solution proves to be effective in meeting the Euclid accuracy requirements for most of the redshift range of the survey. Finally, we investigate the impact on the SED reconstruction of a new set of 16 evenly-spaced medium-band filters for the Subaru telescope, providing quasi-spectroscopic coverage of the VIS passband. This study shows promising results, ensuring accurate SED reconstruction and meeting the mission PSF requirements. This work thus provides not only the methodological foundation of galaxy SED reconstruction in the Euclid PHZ PF, but also a roadmap for future improvements using a new medium-band survey.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.11127-b31b1b.svg)](https://arxiv.org/abs/2601.11127) | **Molecular gas and star formation in central rings across nearby galaxies**  |
|| D. R. Gleis, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>J. Neumann</mark> |
|*Appeared on*| *2026-01-19*|
|*Comments*| *17 pages, 10 figures, accepted for publication at A&A*|
|**Abstract**|            Nearby galaxies exhibit a variety of structures, including central rings, similar to the MW Central Molecular Zone (CMZ). These rings are common in barred galaxies and can be gas-rich and highly star-forming. We aim to study molecular gas content and star formation rate of central rings within nearby galaxies and link them to global galaxy properties (e.g. bar morphology). We utilize $1\,$'' resolution CO(2-1) PHANGS-ALMA observations, visually identify 20 central rings and determine their properties. For $14$ rings, SFR surface density maps are available. We derive ring geometry, integrated molecular gas masses, SFRs, depletion times, and compare them to host galaxy and bar properties. Molecular gas is a good tracer for central rings: Previous studies used ionized gas and dust tracers to identify central rings in galaxies of similar morphological types as this study. In comparison, we find similar fractions of galaxies hosting central rings and similar radii distributions. The gaseous central rings have typical radii of $400_{-150}^{+250}\,$pc, molecular gas masses of $\log(M_\text{mol}/M_\odot){\sim}8.1_{-0.23}^{+0.17}$, and SFRs of $0.21_{-0.16}^{+0.15}\,M_\odot/\text{yr}$, thus contributing $5.6_{-2.1}^{+4.5}\,\%$ and $13_{-5}^{+10}\,\%$ to their host galaxies' molecular gas mass and SFR. The MW CMZ sits at the lower end of the radius, molecular gas mass, and SFR distribution, but it has a similar molecular gas mass and SFR fraction, and depletion time. Longer bars contain more massive molecular central rings, but we find no correlation between bar strength and the ring's molecular gas content. Although absolute central ring properties likely depend on host galaxy properties, the similarities between the MW CMZ and PHANGS central rings in relative parameters suggest that the processes of gas inflow and star formation are similar for central rings across nearby galaxies.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.11469-b31b1b.svg)](https://arxiv.org/abs/2601.11469) | **KMT-2025-BLG-1616Lb: First Microlensing Bound Planet From DREAMS**  |
|| H. Yang, et al. |
|*Appeared on*| *2026-01-19*|
|*Comments*| *Accepted for publication in AJ*|
|**Abstract**|            We present observations and analysis of the bound planetary microlensing event KMT-2025-BLG-1616. The planetary signal was captured by the Korea Microlensing Telescope Network (KMTNet) and the DECam Rogue Earths and Mars Survey (DREAMS). DREAMS's minute-cadence observations break the central/resonant degeneracy in the binary-lens models. The color of the faint source star ($I=22$) is measured from the DREAMS's $r - z$ color. The planetary system has a planet-host mass ratio of $q \sim 5 \times 10^{-4}$. A Bayesian analysis yields a host-star mass of $\sim 0.3\,M_\odot$, a planetary mass of $\sim 40\,M_{\oplus}$, a projected planet-host separation of $\sim 1.6~\mathrm{au}$, and a lens distance of $\sim 7.5~\mathrm{kpc}$. Based on the photometric precision achieved by DREAMS for this event, we simulate free-floating planet (FFP) detections and find that DREAMS is sensitive to Mars-mass FFPs in the Galactic bulge and Moon-mass FFPs in the Galactic disk.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.11280-b31b1b.svg)](https://arxiv.org/abs/2601.11280) | **Detection of four cold Jupiters through combined analyses of radial velocity and astrometry data**  |
|| <mark>Y. Wu</mark>, et al. |
|*Appeared on*| *2026-01-19*|
|*Comments*| *28 pages, 21 figures*|
|**Abstract**|            Cold Jupiters play a crucial role in planet formation and dynamical evolution. Since their initial discovery around 47 UMa, they have attracted significant interest, yet their formation mechanisms remain uncertain, underscoring the need to expand the known population. In this work, we combine RV data with Gaia astrometry using Hipparcos-Gaia proper-motion anomalies over a 25-year baseline. By jointly modeling both datasets with the MCMC framework, we constrain planetary masses, orbital inclinations, and three-dimensional orbital architectures. This reduces RV degeneracies and improves mass determinations. Four cold Jupiters are reported: HD 68475 b and HD 100508 b are each the first confirmed planet in their systems, with orbital periods $7832_{-323}^{+463}$ d and $5681\pm42$ d and dynamical masses of $5.16_{-0.47}^{+0.53} M_{\text{Jup}}$ and $1.2_{-0.18}^{+0.30} M_{\text{Jup}}$, respectively. In multi-planet systems, HD 48265 c has a period of $10418_{-1400}^{+2400}$ d and a mass of $3.71_{-0.43}^{+0.68} M_{\text{Jup}}$, while HD 114386 c orbits at $444.00_{-0.88}^{+0.93}$ d with a minimum mass of $0.37 \pm 0.03 M_{\text{Jup}}$. The two planets in the HD 48265 system may exhibit a significant mutual inclination, making it a target for testing the von-Zeipel-Kozai-Lidov mechanism. HD 68475 b is a promising candidate for future direct imaging with ELT/METIS. We identified a Jupiter analog with the longest known orbital period among planets with masses between 0.5 and 2 $M_{\text{Jup}}$, implying that a substantial population of cold Jupiters likely awaits discovery by Gaia. This study expands the sample of cold Jupiters with constrained orbits and dynamical masses, demonstrating the value of combining radial velocity and astrometry in exoplanet research.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2601.10795/./Figures/07.png', 'tmp_2601.10795/./Figures/01.png', 'tmp_2601.10795/./Figures/06.png']
copying  tmp_2601.10795/./Figures/07.png to _build/html/
copying  tmp_2601.10795/./Figures/01.png to _build/html/
copying  tmp_2601.10795/./Figures/06.png to _build/html/
exported in  _build/html/2601.10795.md
    + _build/html/tmp_2601.10795/./Figures/07.png
    + _build/html/tmp_2601.10795/./Figures/01.png
    + _build/html/tmp_2601.10795/./Figures/06.png
found figures ['tmp_2601.11127/./plots/ring-sample_on_SFMS_4.png', 'tmp_2601.11127/./plots/ring_gallery.png', 'tmp_2601.11127/./plots/distr-violins_4.png']
copying  tmp_2601.11127/./plots/ring-sample_on_SFMS_4.png to _build/html/
copying  tmp_2601.11127/./plots/ring_gallery.png to _build/html/
copying  tmp_2601.11127/./plots/distr-violins_4.png to _build/html/
exported in  _build/html/2601.11127.md
    + _build/html/tmp_2601.11127/./plots/ring-sample_on_SFMS_4.png
    + _build/html/tmp_2601.11127/./plots/ring_galle

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\jheap}{JHEAp}$
$\newcommand{\joss}{JOSS}$
$\newcommand{\IJMPD}{IJMPD}$
$\newcommand{\phil}{Phil. Trans. R. Soc. A}$
$\newcommand{\nuclearreview}{Nucl. Phys. Rev.}$
$\newcommand{\diff}{\mathop \!\mathrm{d}}$
$\newcommand{\orcid}[1]$
$\newcommand{\refer}[1]{,\ #1; [{\tt #1}]}$
$\newcommand{\itemm}{$
$\medskip\noindent}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand\aa{@extracttenauthor#1}$
$\newcommand\aa{@splitninth#1}$
$\newcommand\aa{@ninthauthor}$
$\newcommand$</div>



<div id="title">

# $\Euclid$: Galaxy SED reconstruction in the PHZ processing function: impact on the PSF and the role of medium-band filters

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.10795-b31b1b.svg)](https://arxiv.org/abs/2601.10795)<mark>Appeared on: 2026-01-19</mark> -  _17 pages, 10 figures, 2 tables, including appendices_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** $\iffalse{\colour{blue} \bf NEW VERSION: We have a modified style file,     aaEC.cls, used here; with that, the bug reported in Sect. 8 no     longer occurs. In order to demonstrate this, the long author list     of [Cuillandre, et. al (2025)](https://doi.org/10.1051/0004-6361/202450803) was copied here.}$ \ $\fi$ Weak lensing surveys require accurate correction for the point spread function (PSF) when measuring galaxy shapes. For a diffraction-limited PSF, as arises in space-based missions, this correction depends on each galaxy’s spectral energy distribution (SED), hence a sufficiently accurate knowledge of the SED is needed for every galaxy. In the $\Euclid$ mission, galaxy SED reconstruction, which is one of the tasks of the photometric-redshift processing function (PHZ PF), relies on broad- and medium-band ancillary photometry. The limited wavelength sampling of the $\IE$ passband and signal-to-noise ratio may affect the reconstruction accuracy and translate into biases in the weak lensing measurements.In this study, we present the methodology, which is employed in the $\Euclid$ PHZ PF, for reconstructing galaxy SEDs at 55 wavelengths, sampling the $\IE$ passband every 10 nm, and we assess whether it fulfils the accuracy requirements imposed on the $\Euclid$ PSF model.We employ both physics- and data-driven methods, focusing on a new approach of template-based flux correction and Gaussian processes. For validation, we introduce an SED metric whose bias translates into inaccuracies in the PSF quadrupole moments.Our findings demonstrate that Gaussian processes and template fitting meet the requirements only in specific, but complementary, redshift intervals. We therefore propose a hybrid approach, which leverages both methods. This solution proves to be effective in meeting the $\Euclid$ accuracy requirements for most of the redshift range of the survey. Finally, we investigate the impact on the SED reconstruction of a new set of 16 evenly-spaced medium-band filters for the Subaru telescope, providing quasi-spectroscopic coverage of the $\IE$ passband. This study shows promising results, ensuring accurate SED reconstruction and meeting the mission PSF requirements. This work thus provides not only the methodological foundation of galaxy SED reconstruction in the $\Euclid$ PHZ PF, but also a roadmap for future improvements using a new medium-band survey. $\iffalse$ Weak lensing surveys require accurate correction for the point spread function (PSF) when measuring galaxy shapes. For a diffraction-limited PSF, as arises in space-based missions, this correction depends on each galaxy’s spectral energy distribution (SED), hence a sufficiently accurate knowledge of the SED is needed for every galaxy. In the $\Euclid$ mission, galaxy SED reconstruction, which is one of the tasks of the photometric-redshift processing function (PHZ PF), relies on broad- and medium-band ancillary photometry. The limited wavelength sampling of the $\IE$ passband and signal-to-noise ratio may affect the reconstruction accuracy and translate into biases in the weak lensing measurements.In this study, we present the methodology, which is employed in the $\Euclid$ PHZ PF, for reconstructing galaxy SEDs in 55 points, sampling the $\IE$ passband every 10 nm, and we assess whether it fulfils the accuracy requirements imposed on the $\Euclid$ PSF model.We employ both physics- and data-driven methods, focusing on a new approach of template-based flux correction and Gaussian processes. For validation, we introduce an SED metric whose bias translates into inaccuracies in the PSF quadrupole moments.Our findings demonstrate that Gaussian processes and template fitting meet the requirements only in specific, but complementary, redshift intervals. We therefore propose a hybrid approach, which leverages both methods. This solution proves to be effective in meeting the $\Euclid$ accuracy requirements across the redshift range of the survey. Finally, we investigate the impact on the SED reconstruction of a new set of 16 evenly-spaced medium-band filters for the Subaru telescope, providing quasi-spectroscopic coverage of the $\IE$ passband. This study shows promising results, ensuring accurate SED reconstruction and meeting the mission PSF requirements. This work thus provides not only the methodological foundation of galaxy SED reconstruction in the $\Euclid$ PHZ PF, but also a roadmap for future improvements using a new medium-band survey. $\fi$

</div>

<div id="div_fig1">

<img src="tmp_2601.10795/./Figures/07.png" alt="Fig6" width="100%"/>

**Figure 6. -** 
   The new HSC medium-band filter design, with 16 filters organised in four quadrants (MBQ1--4). Each filter is named after its weighted central wavelength, indicated by a dashed vertical line. The \IE filter is shown in the background. (*fig:07*)

</div>
<div id="div_fig2">

<img src="tmp_2601.10795/./Figures/01.png" alt="Fig1" width="100%"/>

**Figure 1. -** Median fractional bias in the SED metric arising from the interpolation of galaxy SEDs from coarse sampling down to 1 nm sampling using linear (pink line) and cubic spline (dark green) interpolation. The dotted line represents the \Euclid requirement set on the PSF model. (*fig:01*)

</div>
<div id="div_fig3">

<img src="tmp_2601.10795/./Figures/06.png" alt="Fig5" width="100%"/>

**Figure 5. -** Bias in the SED metric averaged over single redshift bins, calculated from the SED reconstruction using linear (sand-coloured) and b-spline (in brown) interpolation, respectively. (*fig:06*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.10795"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\MPIA}{\label{MPIA} Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{\NAOJ}{\label{NAOJ} National Astronomical Observatory of Japan, 2-21-1 Osawa, Mitaka, Tokyo 181-8588, Japan}$
$\newcommand{\UGent}{\label{UGent} Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\OAN}{\label{OAN} Observatorio Astronómico Nacional (IGN), C/Alfonso XII, 3, E-28014 Madrid, Spain}$
$\newcommand{\ESO}{\label{ESO} European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany}$
$\newcommand{\ULyon}{\label{ULyon} Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574,\ F-69230 Saint-Genis-Laval, France}$
$\newcommand{\OSU}{\label{OSU} Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}$
$\newcommand{\UBonn}{\label{UBonn} Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}$
$\newcommand{\Alberta}{\label{Alberta} Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}$
$\newcommand{\ITA}{\label{ITA} Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\newcommand{\COOL}{\label{COOL}{Cosmic Origins Of Life (COOL) Research DAO, \href{https://coolresearch.io}{https://coolresearch.io}}}$
$\newcommand{\UWyoming}{\label{UWyoming} Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\newcommand{\ANU}{\label{ANU} Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\IWR}{\label{IWR} Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}$
$\newcommand{\CfA}{\label{CfA} Harvard-Smithsonian Center for Astrophysics, 60 Garden Street, Cambridge, MA 02138, USA}$
$\newcommand{\Radcliffe}{\label{Radcliffe}{Elizabeth S. and Richard M. Cashin Fellow at the Radcliffe Institute for Advanced Studies at Harvard University, 10 Garden Street, Cambridge, MA 02138, USA}}$
$\newcommand{\STScI}{\label{STScI} Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\TKU}{\label{TKU} Department of Physics, Tamkang University, No.151, Yingzhuan Rd., Tamsui Dist., New Taipei City 251301, Taiwan}$
$\newcommand{\UCM}{\label{UCM} Departamento de Física de la Tierra y Astrofísica, Universidad Complutense de Madrid, E-28040, Spain}$
$\newcommand{\Princeton}{\label{Princeton} Department of Astrophysical Sciences, Princeton University, 4 Ivy Ln., Princeton, NJ 08544 USA}$
$\newcommand{\Maryland}{\label{Maryland} Department of Astronomy, University of Maryland, 4296 Stadium Drive, College Park, MD 20742, USA}$
$\newcommand{\Oxford}{\label{Oxford} Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{\insubria}{ \label{insubria} Como Lake centre for AstroPhysics (CLAP), DiSAT, Universit{à} dell’Insubria, via Valleggio 11, 22100 Como, Italy}$
$\newcommand{\sectionautorefname}{Section}$
$\newcommand{\subsectionautorefname}{Section}$
$\newcommand{\subsubsectionautorefname}{Section}$</div>



<div id="title">

# Molecular gas and star formation in central rings across nearby galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.11127-b31b1b.svg)](https://arxiv.org/abs/2601.11127)<mark>Appeared on: 2026-01-19</mark> -  _17 pages, 10 figures, accepted for publication at A&A_

</div>
<div id="authors">

D. R. Gleis, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>J. Neumann</mark>

</div>
<div id="abstract">

**Abstract:** Nearby galaxies exhibit a variety of structures, including so-called central or (circum-)nuclear rings that are similar to the Milky Way (MW) Central Molecular Zone (CMZ). These rings are common in barred galaxies and can be gas-rich and highly star-forming. We aim to study the molecular gas content and star formation rate of central rings within nearby galaxies and link them to global galaxy properties -- especially  the bar morphology. We utilize $1 $ $\arcsec$ ( ${\lesssim} 100 $ pc) resolution CO(2-1) observations from the PHANGS-ALMA survey, visually identify 20 central rings and determine their properties. For $14$ of these rings, MUSE observations tracing star formation rate (SFR) surface density are available. We derive the rings' geometry, integrated molecular gas masses, SFRs, depletion times, and compare them to host galaxy and bar properties from the literature. Molecular gas is an effective tracer for central rings: Previous studies used ionized gas and dust tracers to identify central rings in galaxies of similar morphological types as the PHANGS galaxies (numerical Hubble type $T{\sim}-3$ to $T{\sim}9$ ). Compared with those, molecular gas yields similar  fractions of galaxies hosting central rings and similar radii distributions. The gaseous central rings have typical radii of ${\sim} 400_{-150}^{+250} $ pc, molecular gas masses of $\log(M_\text{mol}/M_\odot){\sim}8.1_{-0.23}^{+0.17}$ , and SFRs of ${\sim} 0.21_{-0.16}^{+0.15} M_\odot/\text{yr}$ , thus contributing $5.6_{-2.1}^{+4.5} \%$ and $13_{-5}^{+10} \%$ to their host galaxies' molecular gas mass and SFR, respectively. While the MW CMZ sits at the lower end of the radius, molecular gas mass, and SFR distribution, it matches well in terms of ring molecular gas mass and SFR fraction, and depletion time. Longer bars contain more massive molecular central rings, but there is no correlation between the classical bar strength parameters ( $Q_\text{b}$ , $\varepsilon_\text{bar}$ , $A_2^\text{max}$ ) and the ring's molecular gas content. Although absolute central ring properties (ring radius, molecular gas mass, SFR) likely depend on host galaxy properties, the similarities between the MW CMZ and PHANGS central rings in relative parameters (molecular gas and SFR fraction, depletion time) suggest that the processes of gas inflow and star formation are similar for central rings across nearby galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2601.11127/./plots/ring-sample_on_SFMS_4.png" alt="Fig7" width="100%"/>

**Figure 7. -** Global SFR as a function of global stellar mass for our sample galaxies from the PHANGS-ALMA sample. The solid and dashed black lines denote the main sequence of star forming galaxies with a scatter of 0.36 dex, as given by [Leroy, et. al (2019)](https://dx.doi.org/10.3847/1538-4365/ab3925). The PHANGS galaxies with central rings are color coded by the central ring gas mass as obtained in this study and the different symbols denote barred spiral (diamond) and unbarred spiral (open diamond) galaxies, barred early-type galaxies (ETGs, cross) and unbarred ETGs (open cross). The MW is plotted as a black dot with red errorbars. (*fig:rings_on_SFMS*)

</div>
<div id="div_fig2">

<img src="tmp_2601.11127/./plots/ring_gallery.png" alt="Fig8" width="100%"/>

**Figure 8. -** Gallery of central rings identified in a sample of 81 PHANGS-ALMA galaxies. The galaxy name is given at the top of each panel. All panels have the same physical size of $3 \times 3 \text{kpc}^2$, but with differing color stretches. The minimum and maximum intensity is listed at the bottom right of each panel. The "best" elliptical annulus (see Sect. \ref{sec:ring-shapes}) is plotted in red and the beam size is plotted in white in the lower left corner. (*fig:rings_gallery*)

</div>
<div id="div_fig3">

<img src="tmp_2601.11127/./plots/distr-violins_4.png" alt="Fig9" width="100%"/>

**Figure 9. -** Distributions of central ring parameters (from left to right): radius $r_\text{ring}$, relative radius $r_\text{ring}/r_{25}$, molecular gas mass $M_\text{mol}^\text{ring}$, ring molecular gas mass fraction $M_\text{mol}^\text{ring}/ M_\text{mol}^\text{gal}$, star formation rate ${\rm  SFR}_\text{ring}$, ring SFR fraction ${\rm SFR}_\text{ring} / {\rm SFR}_\text{gal}$, depletion time $M_\text{mol}^\text{ring} / {\rm SFR}_\text{ring}$, and relative depletion time $t_\text{dep}^\text{ring} / t_\text{dep}^\text{gal}$. The distributions are shown for the whole sample on the left and for spirals only on the right, thus leading to the different numbers $N= \{...\}$ considered for the distributions. The violins show a smoothed distribution, the boxes contain the $25$th to $75$th percentile of the distributions, the black whiskers span out to $1.5$ times the interquartile range, and the circles mark outliers beyond this range. The median is shown by the black horizontal line. The respective values for the MW CMZ are plotted as black diamonds along with their error bars in red. (*fig:violin-distr*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.11127"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thetae}{\theta_{\rm E}}$
$\newcommand{\teff}{t_{\rm eff}}$
$\newcommand{\Icat}{I_{\rm cat}}$
$\newcommand{\pie}{\pi_{\rm E}}$
$\newcommand{\pirel}{\pi_{\rm rel}}$
$\newcommand{\te}{t_{\rm E}}$
$\newcommand{\event}{KMT-2025-BLG-1616}$
$\newcommand{\an}{\theta_{*}}$
$\newcommand{\Sp}{{\it Spitzer}}$
$\newcommand{\hjd}{{\rm HJD}^{\prime}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# ${\large KMT-2025-BLG-1616Lb: First Microlensing Bound  Planet From DREAMS}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.11469-b31b1b.svg)](https://arxiv.org/abs/2601.11469)<mark>Appeared on: 2026-01-19</mark> -  _Accepted for publication in AJ_

</div>
<div id="authors">

H. Yang (杨弘靖), et al.

</div>
<div id="abstract">

**Abstract:** We present observations and analysis of the bound planetary microlensing event KMT-2025-BLG-1616. The planetary signal was captured by the Korea Microlensing Telescope Network (KMTNet) and the DECam Rogue Earths and Mars Survey (DREAMS). DREAMS's minute-cadence observations break the central/resonant degeneracy in the binary-lens models. The color of the faint source star ( $I=22$ ) is measured from the DREAMS's $r - z$ color. The planetary system has a planet-host mass ratio of $q \sim 5 \times 10^{-4}$ . A Bayesian analysis yields a host-star mass of $\sim 0.3 M_\odot$ , a planetary mass of $\sim 40 M_{\oplus}$ , a projected planet-host separation of $\sim 1.6 \mathrm{au}$ , and a lens distance of $\sim 7.5 \mathrm{kpc}$ . Based on the photometric precision achieved by DREAMS for this event, we simulate free-floating planet (FFP) detections and find that DREAMS is sensitive to Mars-mass FFPs in the Galactic bulge and Moon-mass FFPs in the Galactic disk.

</div>

<div id="div_fig1">

<img src="tmp_2601.11469/./Figure/KB251616-lc.png" alt="Fig4" width="100%"/>

**Figure 4. -** Light curve of the microlensing event, $\event$, with the 2L1S models (solid black and orange lines) and the underlying 1L1S model (dashed grey line). Different data sets are plotted in different colors. The upper panels shows the 8-day data around the peak. The lower panels present a close-up of the planetary anomaly and the residuals relative to the 2L1S models. (*fig:lc*)

</div>
<div id="div_fig2">

<img src="tmp_2601.11469/./Figure/KB251616_gridsearch.png" alt="Fig1" width="100%"/>

**Figure 1. -** The $\chi^2$ surface in the $(\log s, \log q)$ plane obtained from the 2L1S grid search. The upper panel shows the coarse grid, while the lower panel zooms in on the dense grid around the global minimum. Dark red, red, yellow, green, blue, and dark blue denote grid points within $\Delta\chi^2<1n$, $<2n$, $<3n$, $<4n$, $<5n$, and $<6n$, respectively, where $n = 80$. Grid points with $\Delta\chi^2 > 480$ are left blank. The labels "Close Central", "Wide Central", "Close Resonant", and "Wide Resonant" in the lower panel mark the four local minima. (*fig:grid*)

</div>
<div id="div_fig3">

<img src="tmp_2601.11469/./Figure/KB251616_CMD.png" alt="Fig3" width="100%"/>

**Figure 3. -** Color-magnitude diagram for $\event$. Black dots show OGLE-III field stars  ([Szyma\'nski, et. al 2011](http://adsabs.harvard.edu/abs/2011AcA....61...83S))  within a $1.5'$ from the event position. The red asterisk and blue dot mark the centroid of the red clump and the source star, respectively. Green points show the _HST_ CMD from [Holtzman, et. al (1998)](http://adsabs.harvard.edu/abs/1998AJ....115.1946H), and the HST red clump centroid, $(V-I,I)_{\rm cl,HST}=(1.62,15.15)$ ([Bennett, et. al 2008](http://adsabs.harvard.edu/abs/2008ApJ...684..663B)) , has been shifted to match the OGLE-III red clump. (*fig:cmd*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.11469"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

114  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
